## 1️⃣ Setup - Clone Repository & Install Dependencies

In [ ]:
# Clone the repository
!git clone https://github.com/ArshvirSk/Multimodal-Rare-Disease.git
%cd Multimodal-Rare-Disease

In [ ]:
# Install dependencies
!pip install -q torch torchvision transformers pillow tqdm matplotlib seaborn

In [ ]:
# Verify GPU is available
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2️⃣ Generate Augmented Training Data

Expands 50 base images → 550 augmented images

In [ ]:
# Generate augmented images
!python scripts/generate_augmented_data.py

In [ ]:
# Verify augmented data
import os
from pathlib import Path

aug_dir = Path("data/images_augmented")
total_images = sum(1 for _ in aug_dir.rglob("*.png")) + sum(1 for _ in aug_dir.rglob("*.jpg"))
syndromes = [d.name for d in aug_dir.iterdir() if d.is_dir()]

print(f"✅ Total augmented images: {total_images}")
print(f"✅ Syndromes: {len(syndromes)}")
for s in syndromes:
    count = len(list((aug_dir / s).glob("*")))
    print(f"   • {s}: {count} images")

## 3️⃣ Train the Model

Training with:
- **Backbone:** ResNet50 (pretrained on ImageNet)
- **Transfer Learning:** Layers 1-3 frozen
- **Regularization:** Dropout 0.6, Label smoothing 0.12
- **Optimizer:** AdamW with Cosine Annealing LR

In [ ]:
# Train the model (takes ~10-15 minutes on T4 GPU)
!python run_training.py --image-dirs data/images_augmented --epochs 60 --aug-factor 4 --batch-size 16

## 4️⃣ Make Predictions

Test the trained model on sample images

In [ ]:
# Test prediction on a sample image
!python predict.py --image data/images/SYN_AS_001.png

In [ ]:
# Test on multiple images
import glob

# Get one image from each syndrome
test_images = [
    "data/images/SYN_CdLS_001.png",  # Cornelia de Lange
    "data/images/SYN_WBS_001.png",   # Williams-Beuren
    "data/images/SYN_NS_001.png",    # Noonan
    "data/images/SYN_KS_001.png",    # Kabuki
]

for img in test_images:
    print(f"\n{'='*60}")
    !python predict.py --image {img}

## 5️⃣ Upload Your Own Image

Upload a facial image to classify

In [ ]:
# Upload your own image
from google.colab import files
from IPython.display import Image, display

print("📤 Upload a facial image (JPG/PNG):")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    print(f"\n✅ Uploaded: {filename}")
    
    # Display the image
    display(Image(filename, width=300))
    
    # Run prediction
    print("\n🔍 Running prediction...\n")
    !python predict.py --image "{filename}"

## 6️⃣ Visualize Results

In [ ]:
# Visualize sample images from each syndrome
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Sample Images from Each Syndrome', fontsize=14)

syndromes = [
    ("SYN_CdLS", "Cornelia de Lange"),
    ("SYN_WBS", "Williams-Beuren"),
    ("SYN_NS", "Noonan"),
    ("SYN_KS", "Kabuki"),
    ("SYN_KBG", "KBG"),
    ("SYN_AS", "Angelman"),
    ("SYN_RSTS", "Rubinstein-Taybi"),
    ("SYN_SMS", "Smith-Magenis"),
    ("SYN_NBS", "Nicolaides-Baraitser"),
    ("SYN_22Q", "22q11.2 Deletion"),
]

for idx, (code, name) in enumerate(syndromes):
    ax = axes[idx // 5, idx % 5]
    img_path = f"data/images/{code}_001.png"
    if Path(img_path).exists():
        img = Image.open(img_path)
        ax.imshow(img)
    ax.set_title(name, fontsize=9)
    ax.axis('off')

plt.tight_layout()
plt.show()

## 📊 Model Architecture

```
┌─────────────────────────────────────────┐
│           Input Image (224x224)         │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│     ResNet50 Backbone (ImageNet)        │
│     • Layers 1-3: Frozen                │
│     • Layer 4: Trainable                │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│         Feature Vector (2048-d)         │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│    Classification Head + Dropout        │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│      Softmax → 10 Syndrome Classes      │
└─────────────────────────────────────────┘
```

---

## 🔗 Links

- **GitHub:** https://github.com/ArshvirSk/Multimodal-Rare-Disease
- **PDIDB Dataset:** https://github.com/WGLab/PDIDB
- **Orphadata:** https://www.orphadata.com

---

*Made with ❤️ for rare disease research*